# Neural Networks for Tokens data

We saw that most of our base models did not perform well in terms of precision of the target class.

Neural Networks are a powerful machine learning tool, we expect them to outperform the base models in our use case.

We saw that we had extremely low coefficients with our target class in our eda so we already knew we needed an in depth and powerful model. Neural Networks are great at capturing complex and non-linear relationships. However, with their power comes a slight drawback, the element of random chance. 

We've taken this into account, and made precautions to prevent both overfitting and a one off "lucky" successful model.

While conducting a test/train split is a good method in ensuring a model performs well with unseen data, we took extra precaution. We tested our models both on raw data as well as pca data and ensured the results were within a reasonable range. A good model should detect the underlying pattern in any representation of the same data. PCA helps mitigate collinearity within our data as well as overfitting to the raw train data.

We also took an aggregate measure of our model evaluations. Each models architecure was trained multiple times on multiple variations of data. Every single run, for every single model architecure regardless if it was pca data or raw data, was profitable at one or more thresholds. This validates all of our model architures, and suggests in a real world setting we can expect results within our range of results. Therefore we can rule out profitability being due to luck and overfitting. 

Table of Contents:<a id="Contents"></a>
* [Base Model Architecture](#model1)
* [Second Model Architecture](#model2)
* [Complex Model Architecture](#model3)
* [Model Evaluation](#modeleval)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pickle

# Neural Networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight

# Model Evaluation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
df = pd.read_csv('../data/complete_df.csv')

We must remove metrics from our X that were used to determine the target class and will not be avaiable to us when predicting new tokens

In [3]:
X = df.drop(columns=['suitable_investment', 'ath_marketcap', 'growth', 'ath_price']).select_dtypes(include=['number', 'bool'])
y = df['suitable_investment']

X = X.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11, stratify=y)

In [4]:
print(f'Our target class accounts for {y_train.sum() / len(y_train)}% of our train data')
print(f'Our target class accounts for {y_test.sum() / len(y_test)}% of our test data')

Our target class accounts for 0.011168557935495423% of our train data
Our target class accounts for 0.01114617605586517% of our test data


As an extra measure of model validity, we're going to test the model on original data, pca data, and collinearity adjusted data

In [5]:
my_PCA = PCA()
my_PCA.fit(X_train)
X_train_PCA = my_PCA.transform(X_train)
X_test_PCA = my_PCA.transform(X_test)

Given our imbalanced data, we need to ensure we are running our neural network with weighted classes

In [6]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Model 1<a id="model1"></a>
<p><a href="#Contents" style="font-size: 12px;">Back to Table of Contents</a></p>

In [59]:
tf.random.set_seed(123)

model1_5pca = keras.Sequential()

regularizer1 = keras.regularizers.l2(0.001)

model1_5pca.add(layers.BatchNormalization())
model1_5pca.add(layers.Dense(64, activation="relu", kernel_regularizer=regularizer1))
model1_5pca.add(layers.Dropout(0.3))
model1_5pca.add(layers.Dense(128, activation="relu", kernel_regularizer=regularizer1))
model1_5pca.add(layers.BatchNormalization())
model1_5pca.add(layers.Dense(256, activation="relu", kernel_regularizer=regularizer1))
model1_5pca.add(layers.Dropout(0.3))
model1_5pca.add(layers.Dense(256, activation="relu", kernel_regularizer=regularizer1))
model1_5pca.add(layers.BatchNormalization())
model1_5pca.add(layers.Dense(128, activation="relu", kernel_regularizer=regularizer1))
model1_5pca.add(layers.Dropout(0.3))
model1_5pca.add(layers.Dense(64, activation="relu", kernel_regularizer=regularizer1))
model1_5pca.add(layers.BatchNormalization())
model1_5pca.add(layers.Dense(32, activation="relu", kernel_regularizer=regularizer1))
model1_5pca.add(layers.Dense(16, activation="relu", kernel_regularizer=regularizer1))
model1_5pca.add(layers.Dense(8, activation="relu", kernel_regularizer=regularizer1))

model1_5pca.add(layers.Dense(1, activation="sigmoid"))

model1_5pca.compile(
    optimizer=keras.optimizers.Adam(),  
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()]
)

In [60]:
# 10 epochs
model1_5pca.fit(X_train_PCA, y_train, epochs=25, verbose=0, class_weight=class_weights_dict)

Threshold optimization to maximize precision

In [186]:
y_pred = model1_5pca.predict(X_test_PCA)

for threshold in np.arange(0.7, 0.9, 0.05):
    y_pred_binary = (y_pred >= threshold).astype(int)

    # Evaluate model performance at each threshold
    print(threshold)
    print(confusion_matrix(y_test, y_pred_binary))
    print(classification_report(y_test, y_pred_binary))
    print('-------------------------------------------------')

466/466 [==============================] - 0s 642us/step
0.7
[[14285   442]
 [  140    26]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     14727
           1       0.06      0.16      0.08       166

    accuracy                           0.96     14893
   macro avg       0.52      0.56      0.53     14893
weighted avg       0.98      0.96      0.97     14893

-------------------------------------------------
0.75
[[14524   203]
 [  150    16]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     14727
           1       0.07      0.10      0.08       166

    accuracy                           0.98     14893
   macro avg       0.53      0.54      0.54     14893
weighted avg       0.98      0.98      0.98     14893

-------------------------------------------------
0.8
[[14619   108]
 [  157     9]]
              precision    recall  f1-score   support

           0       0.99     

#### Model 1 findings 
tp/fp @ threshold (precision, recall):

original data (25 epochs):
1. model1_1 | 13/161 @ .7 (7, 8) ----  6/61 @ .75 (9, 4) ---- 1/28 @ .8 (3, 1) ---- 0/10 @ .85 (0, 0)
2. model1_2 | 17/240 @ .7 (7, 10) ----  7/91 @ .75 (7, 4) ---- 3/49 @ .8 (6, 2) ---- 0/16 @ .85 (0, 0)
3. model1_3 | 27/490 @ .7 (5, 16) ---- 13/156 @ .75 (8, 8) ---- 7/70 @ .8 (9, 4) ---- 3/27 @ .85 (10, 2)
4. model1_4 | 18/306 @ .7 (6, 11) ---- 9/78 @ .75 (10, 5) ---- 1/33 @ .8 (3, 1) ---- 0/10 @ .85 (0, 0)
5. model1_5 | 18/244 @ .7 (7, 11) ---- 7/60 @ .75 (10, 4) ---- 1/21 @ .8 (5, 1) ---- 0/7 @ .85 (0, 0)

pca data:
1. model1_1pca | 28/448 @ .7 (6, 17) ----  13/206 @ .75 (6, 8) ---- 7/96 @ .8 (7, 4) ---- 5/58 @ .85 (8, 3)
2. model1_2pca | 27/466 @ .7 (5, 16) ----  16/213 @ .75 (7, 10) ---- 9/101 @ .8 (8, 5) ---- 5/53 @ .85 (9, 3)
3. model1_3pca | 31/525 @ .7 (6, 19) ----  16/181 @ .75 (8, 10) ---- 8/82 @ .8 (9, 5) ---- 5/28 @ .85 (15, 3)
4. model1_4pca | 35/753 @ .7 (4, 21) ----  20/258 @ .75 (7, 12) ---- 10/121 @ .8 (8, 6) ---- 6/70 @ .85 (8, 4)
5. model1_5pca | 16/203 @ .7 (7, 10) ----  9/108 @ .75 (8, 5) ---- 7/62 @ .8 (10, 4) ---- 4/28 @ .85 (12, 2)

Let's find our averages to compare our models:

In [193]:
#original data at .75 threshold
precision_1_1 = ((6/67) + (7/98) + (13/169) + (9/87) + (7/67)) / 5
recall_1_1 = ((6/166) + (7/166) + (13/166) + (9/166) + (7/166)) / 5
print(f'Model 1 with original data at a threshold of 0.75 has an average precision of: {precision_1_1} and recall of {recall_1_1}')
print('---')

#pca data at .85 threshold
precision_1_2 = ((5/63) + (5/58) + (5/33) + (6/76) + (4/32)) / 5
recall_1_2 = ((5/166) + (5/166) + (5/166) + (6/166) + (4/166)) / 5
print(f'Model 1 with pca data at a threshold of 0.85 has an average precision of: {precision_1_2} and recall of {recall_1_2}')
print('---')

#pca data at .80 threshold
precision_1_3 = ((7/103) + (9/110) + (8/90) + (10/131) + (7/69)) / 5
recall_1_3 = ((7/166) + (9/166) + (8/82) + (10/166) + (7/166)) / 5
print(f'Model 1 with pca data at a threshold of 0.80 has an average precision of: {precision_1_3} and recall of {recall_1_3}')
print('---')

#pca data at .75 threshold
precision_1_4 = ((13/219) + (16/229) + (16/197) + (20/278) + (9/117)) / 5
recall_1_4 = ((13/166) + (16/166) + (16/166) + (20/166) + (9/166)) / 5
print(f'Model 1 with pca data at a threshold of 0.75 has an average precision of: {precision_1_4} and recall of {recall_1_4}')
print('---')

Model 1 with original data at a threshold of 0.75 has an average precision of: 0.0891659549919972 and recall of 0.050602409638554224
---
Model 1 with pca data at a threshold of 0.85 has an average precision of: 0.10420689917060152 and recall of 0.03012048192771084
---
Model 1 with pca data at a threshold of 0.80 has an average precision of: 0.08329067779610574 and recall of 0.059271231266529535
---
Model 1 with pca data at a threshold of 0.75 has an average precision of: 0.07186270466094252 and recall of 0.0891566265060241
---


# Model 2<a id="model2"></a>
<p><a href="#Contents" style="font-size: 12px;">Back to Table of Contents</a></p>

In [132]:
tf.random.set_seed(123)

model2_8pca = keras.Sequential()

regularizer2 = keras.regularizers.l2(0.0001)

model2_8pca.add(layers.BatchNormalization())
model2_8pca.add(layers.Dense(64, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.Dropout(0.3))
model2_8pca.add(layers.Dense(128, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.BatchNormalization())
model2_8pca.add(layers.Dense(256, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.Dropout(0.3))
model2_8pca.add(layers.Dense(256, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.Dropout(0.3))
model2_8pca.add(layers.Dense(256, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.BatchNormalization())
model2_8pca.add(layers.Dense(128, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.Dropout(0.3))
model2_8pca.add(layers.Dense(64, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.BatchNormalization())
model2_8pca.add(layers.Dense(32, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.Dense(16, activation="relu", kernel_regularizer=regularizer2))
model2_8pca.add(layers.Dense(8, activation="relu", kernel_regularizer=regularizer2))

model2_8pca.add(layers.Dense(1, activation="sigmoid"))

model2_8pca.compile(
    optimizer=keras.optimizers.Adam(),  
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()]
)

In [133]:
# 10 epochs
history2 = model2_8pca.fit(X_train_PCA, y_train, epochs=10, verbose=0, class_weight=class_weights_dict)

In [134]:
y_pred2 = model2_8pca.predict(X_test_PCA)

for threshold in np.arange(0.7, 0.91, 0.0500001):
    y_pred_binary2 = (y_pred2 >= threshold).astype(int)

    # Evaluate model performance at each threshold
    print(threshold)
    print(confusion_matrix(y_test, y_pred_binary2))
    print(classification_report(y_test, y_pred_binary2))
    print('-------------------------------------------------')

466/466 [==============================] - 0s 704us/step
0.7
[[13861   866]
 [  126    40]]
              precision    recall  f1-score   support

           0       0.99      0.94      0.97     14727
           1       0.04      0.24      0.07       166

    accuracy                           0.93     14893
   macro avg       0.52      0.59      0.52     14893
weighted avg       0.98      0.93      0.96     14893

-------------------------------------------------
0.7500001
[[14263   464]
 [  138    28]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     14727
           1       0.06      0.17      0.09       166

    accuracy                           0.96     14893
   macro avg       0.52      0.57      0.53     14893
weighted avg       0.98      0.96      0.97     14893

-------------------------------------------------
0.8000001999999999
[[14483   244]
 [  147    19]]
              precision    recall  f1-score   support

        

#### Model 2 findings 
tp/fp @ threshold (precision, recall):

original data:
1. model2_1 :20 epochs: 27/481 @ .75 (5, 16) ----  13/238 @ .8 (5, 8) ---- 7/100 @ .85 (7, 4) ---- 3/23 @ .9 (12, 2)
2. model2_2 :20 epochs: 16/212 @ .75 (7, 10) ----  8/57 @ .8 (12, 5) ---- 1/24 @ .85 (4, 1) ---- 0/8 @ .9 (0, 0)
3. model2_3 :20 epochs: 16/231 @ .75 (6, 10) ----  10/88 @ .8 (10, 6) ---- 5/34 @ .85 (13, 3) ---- 1/8 @ .9 (11, 1)
4. model2_4 :20 epochs: 19/304 @ .75 (6, 11) ----  13/135 @ .8 (9, 8) ---- 6/54 @ .85 (10, 4) ---- 1/16 @ .9 (6, 1)

5. model2_5 :30 epochs: 13/184 @ .75 (7, 8) ----  7/72 @ .8 (9, 4) ---- 2/23 @ .85 (8, 1) ---- 0/8 @ .9 (0, 0)
6. model2_9 :30 epochs: 8/151 @ .75 (5, 5) ----  5/59 @ .8 (8, 3) ---- 1/11 @ .85 (8, 1) ---- 0/4 @ .9 (0, 0)
7. model2_7 :30 epochs: 12/199 @ .75 (6, 7) ----  8/70 @ .8 (10, 5) ---- 3/19 @ .85 (14, 2) ---- 0/6 @ .9 (0, 0)
8. model2_8 :30 epochs: 14/266 @ .75 (5, 8) ----  10/107 @ .8 (9, 6) ---- 6/31 @ .85 (16, 4) ---- 0/10 @ .9 (0, 0)


pca data:
1. model2_1pca :20 epochs: 20/304 @ .75 (6, 12) ----  10/135 @ .8 (7, 6) ---- 7/63 @ .85 (10, 4) ---- 3/27 @ .9 (10, 2)
2. model2_2pca :20 epochs: 22/392 @ .75 (5, 13) ----  14/189 @ .8 (7, 8) ---- 9/91 @ .85 (9, 5) ---- 4/42 @ .9 (9, 2)
3. model2_3pca :20 epochs: 21/344 @ .75 (6, 13) ----  14/196 @ .8 (7, 8) ---- 10/108 @ .85 (8, 6) ---- 6/69 @ .9 (8, 4)
4. model2_4pca :20 epochs: 36/650 @ .75 (5, 22) ----  23/360 @ .8 (6, 14) ---- 11/178 @ .85 (6, 7) ---- 8/90 @ .9 (8, 5)

5. model2_5pca :10 epochs: 25/388 @ .75 (6, 15) ----  15/191 @ .8 (7, 9) ---- 9/104 @ .85 (8, 5) ---- 4/49 @ .9 (8, 2)
6. model2_6pca :10 epochs: 32/597 @ .75 (5, 19) ----  21/281 @ .8 (7, 13) ---- 13/150 @ .85 (8, 8) ---- 9/80 @ .9 (10, 5)
7. model2_7pca :10 epochs: 33/625 @ .75 (5, 20) ----  23/361 @ .8 (6, 14) ---- 15/193 @ .85 (7, 9) ---- 9/115 @ .9 (7, 5)
8. model2_8pca :10 epochs: 28/464 @ .75 (5, 20) ----  19/244 @ .8 (7, 11) ---- 11/150 @ .85 (7, 7) ---- 8/88 @ .9 (8, 5)


Let's find our averages to compare our models:

In [194]:
#original data at .80 threshold
precision_2_1 = ((13/251) + (8/65) + (10/98) + (13/148) + (7/79) + (5/64) + (8/78) + (10/117)) / 8
recall_2_1 = ((13/166) + (8/166) + (10/166) + (13/166) + (7/166) + (5/166) + (8/166) + (10/166)) / 8
print(f'Model 2 with original data at a threshold of 0.80 has an average precision of: {precision_2_1} and recall of {recall_2_1}')
print('---')

#original data at .85 threshold
precision_2_2 = ((7/107) + (1/25) + (5/39) + (6/40) + (2/25) + (1/12) + (3/22) + (6/37)) / 8
recall_2_2 = ((7/166) + (1/166) + (5/166) + (6/166) + (2/166) + (1/166) + (3/166) + (6/166)) / 8
print(f'Model 2 with original data at a threshold of 0.85 has an average precision of: {precision_2_2} and recall of {recall_2_2}')
print('---')

Model 2 with original data at a threshold of 0.80 has an average precision of: 0.08993939861218092 and recall of 0.05572289156626506
---
Model 2 with original data at a threshold of 0.85 has an average precision of: 0.10568560260149044 and recall of 0.023343373493975906
---


# Model 3<a id="model3"></a>
<p><a href="#Contents" style="font-size: 12px;">Back to Table of Contents</a></p>

In [180]:
tf.random.set_seed(123)

model3_5pca = keras.Sequential()

regularizer3 = keras.regularizers.l2(0.0001)

model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dense(36, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(72, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(144, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(288, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(576, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(1152, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(576, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(288, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(144, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(72, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(36, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(18, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())
model3_5pca.add(layers.Dropout(0.3))
model3_5pca.add(layers.Dense(9, activation="relu", kernel_regularizer=regularizer3))
model3_5pca.add(layers.BatchNormalization())

model3_5pca.add(layers.Dense(1, activation="sigmoid"))

model3_5pca.compile(
    optimizer=keras.optimizers.Adam(),  
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()]
)

In [181]:
history3 = model3_5pca.fit(X_train_PCA, y_train, epochs=10, verbose=1, class_weight=class_weights_dict)

Epoch 1/10
1397/1397 [==============================] - 21s 13ms/step - loss: 1.1295 - binary_accuracy: 0.4674 - precision_53: 0.0109
Epoch 2/10
1397/1397 [==============================] - 18s 13ms/step - loss: 1.0940 - binary_accuracy: 0.4719 - precision_53: 0.0119
Epoch 3/10
1397/1397 [==============================] - 18s 13ms/step - loss: 1.0916 - binary_accuracy: 0.4419 - precision_53: 0.0123
Epoch 4/10
1397/1397 [==============================] - 18s 13ms/step - loss: 1.0827 - binary_accuracy: 0.5176 - precision_53: 0.0138
Epoch 5/10
1397/1397 [==============================] - 18s 13ms/step - loss: 1.0492 - binary_accuracy: 0.4640 - precision_53: 0.0146
Epoch 6/10
1397/1397 [==============================] - 18s 13ms/step - loss: 1.0161 - binary_accuracy: 0.5222 - precision_53: 0.0151
Epoch 7/10
1397/1397 [==============================] - 18s 13ms/step - loss: 0.9707 - binary_accuracy: 0.5798 - precision_53: 0.0169
Epoch 8/10
1397/1397 [==============================] - 18s 13

In [182]:
y_pred_nn3 = model3_5pca.predict(X_test_PCA)

for threshold in np.arange(0.7, 0.91, 0.05):
    y_pred_binary3 = (y_pred_nn3 >= threshold).astype(int)

    # Evaluate model performance at each threshold
    print(threshold)
    print(confusion_matrix(y_test, y_pred_binary3))
    print(classification_report(y_test, y_pred_binary3))
    print('-------------------------------------------------')

466/466 [==============================] - 1s 3ms/step
0.7
[[14385   342]
 [  146    20]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98     14727
           1       0.06      0.12      0.08       166

    accuracy                           0.97     14893
   macro avg       0.52      0.55      0.53     14893
weighted avg       0.98      0.97      0.97     14893

-------------------------------------------------
0.75
[[14544   183]
 [  153    13]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     14727
           1       0.07      0.08      0.07       166

    accuracy                           0.98     14893
   macro avg       0.53      0.53      0.53     14893
weighted avg       0.98      0.98      0.98     14893

-------------------------------------------------
0.8
[[14623   104]
 [  158     8]]
              precision    recall  f1-score   support

           0       0.99      0

#### Model 3 findings 
tp/fp @ threshold (precision, recall):

original data:
1. model3_1 :10 epochs: 14/224 @ .7 (6, 8)  ---- 7/84 @ .75 (8, 4)    ----  7/63 @ .8 (10, 4)   ---- 3/38 @ .85 (7, 2)    ---- 1/28 @ .9 (3, 1)
2. model3_2 :10 epochs: 26/438 @ .7 (6, 16) ---- 22/348 @ .75 (6, 13) ----  21/259 @ .8 (7, 13) ---- 17/205 @ .85 (8, 10) ---- 13/134 @ .9 (9, 8)
3. model3_3 :10 epochs: 21/308 @ .7 (6, 13) ---- 17/207 @ .75 (8, 10) ----  11/107 @ .8 (9, 7)  ---- 2/37 @ .85 (5, 1)    ---- 0/18 @ .9 (0, 0)
4. model3_4 :10 epochs: 30/478 @ .7 (6, 18) ---- 16/222 @ .75 (7, 10) ----  8/96 @ .8 (8, 5)    ---- 4/47 @ .85 (8, 2)    ---- 0/26 @ .9 (0, 0)
5. model3_5 :10 epochs: 29/597 @ .7 (5, 17) ---- 26/497 @ .75 (5, 16) ----  24/371 @ .8 (6, 14) ---- 20/265 @ .85 (7, 12) ---- 13/142 @ .9 (8, 8)


pca data:
1. model3_1pca :10 epochs: 22/309 @ .7 (7, 13)  ---- 14/135 @ .75 (9, 8) ----  7/76 @ .8 (8, 4)  ---- 2/37 @ .85 (5, 1)  ---- 0/14 @ .9 (0, 0)
2. model3_2pca :10 epochs: 25/514 @ .7 (5, 15)  ---- 14/222 @ .75 (6, 8) ----  9/115 @ .8 (7, 5) ---- 5/51 @ .85 (9, 3)  ---- 1/21 @ .9 (5, 1)
3. model3_3pca :10 epochs: 23/537 @ .7 (4, 14)  ---- 14/292 @ .75 (5, 8) ----  8/136 @ .8 (6, 5) ---- 6/50 @ .85 (11, 4) ---- 3/19 @ .9 (14, 2)
4. model3_4pca :10 epochs: 31/758 @ .7 (4, 19)  ---- 13/175 @ .75 (7, 8) ----  8/57 @ .8 (12, 5) ---- 4/24 @ .85 (14, 2) ---- 0/7 @ .9 (0, 0)
5. model3_5pca :10 epochs: 20/342 @ .7 (6, 12)  ---- 13/183 @ .75 (7, 8) ----  8/104 @ .8 (7, 5) ---- 6/75 @ .85 (7, 4)  ---- 3/43 @ .9 (7, 2)


Let's find our averages to compare our models:

In [196]:
#original data at .80 threshold
precision_3_1 = ((7/70) + (21/280) + (11/118) + (8/104) + (24/395)) / 5
recall_3_1 = ((7/166) + (21/166) + (11/166) + (8/166) + (24/166)) / 5
print(f'Model 3 with original data at a threshold of 0.80 has an average precision of: {precision_3_1} and recall of {recall_3_1}')
print('---')

#pca data at .80 threshold
precision_3_2 = ((7/83) + (9/124) + (8/144) + (8/65) + (8/112)) / 5
recall_3_2 = ((7/166) + (9/166) + (8/166) + (8/166) + (8/166)) / 5
print(f'Model 3 with pca data at a threshold of 0.80 has an average precision of: {precision_3_2} and recall of {recall_3_2}')
print('---')

#pca data at .85 threshold
precision_3_3 = ((2/39) + (5/56) + (6/56) + (4/28) + (6/81)) / 5
recall_3_3 = ((2/166) + (5/166) + (6/166) + (4/166) + (6/166)) / 5
print(f'Model 3 with pca data at a threshold of 0.85 has an average precision of: {precision_3_3} and recall of {recall_3_3}')
print('---')

Model 3 with original data at a threshold of 0.80 has an average precision of: 0.08118058191540277 and recall of 0.0855421686746988
---
Model 3 with pca data at a threshold of 0.80 has an average precision of: 0.08139580892398615 and recall of 0.04819277108433735
---
Model 3 with pca data at a threshold of 0.85 has an average precision of: 0.09292836792836792 and recall of 0.027710843373493978
---


### The Precision and Recall trade-off<a id="modeleval"></a>
<p><a href="#Contents" style="font-size: 12px;">Back to Table of Contents</a></p>

Beyond the scope of this bootcamp, will explore what happens in cases where we are not in the target class. The assumption currently is that it's a scam and you lose all your money, we did not consider the potential of it simply not reaching the minimum growth, or pulling out funds before the rug pulls. Even if there's no profit on these tokens, there is a possibility to approach breakeven.

Assumptions:
- non target class is a 100% loss
- target cashout at 15x
- gas fees, liquidity, and transaction limits are non factors
- $100 in each token 

Based on average marketcap and wallet limitations of the target class, we can put up to $1000 in most cases

we need a 6.7% precision to break even with these assumptions

Based on these assumptions, let's decide what our best model is:

Each investment in a target class investment would return $7500 based on our 15x cashout

In [6]:
total_target = 166
investment_amount = 100

In [7]:
def evaluate_model_profit(precision, recall, total_pos, invested):
    tp = recall * total_pos
    fp = tp * ((1 / precision) - 1)

    spent = round((invested * fp), 2)
    made = round(((invested * 15) * tp), 2)
    profit = made - spent
    multiple = round((made / spent), 2)

    print(f'You made ${made} in good investments') 
    print(f'But lost ${spent} in bad investments')
    print(f'For a profit of ${profit} and a portfolio multiple of {multiple}x')

**Model Architecure 1**

In [287]:
evaluate_model_profit(precision_1_1, recall_1_1, total_target, investment_amount)

You made $12600.0 in good investments
But lost $8580.64 in bad investments
For a profit of $4019.3600000000006 and a portfolio multiple of 1.47x


In [288]:
evaluate_model_profit(precision_1_2, recall_1_2, total_target, investment_amount)

You made $7500.0 in good investments
But lost $4298.15 in bad investments
For a profit of $3201.8500000000004 and a portfolio multiple of 1.74x


In [289]:
evaluate_model_profit(precision_1_3, recall_1_3, total_target, investment_amount)

You made $14758.54 in good investments
But lost $10828.97 in bad investments
For a profit of $3929.5700000000015 and a portfolio multiple of 1.36x


In [290]:
evaluate_model_profit(precision_1_4, recall_1_4, total_target, investment_amount)

You made $22200.0 in good investments
But lost $19114.83 in bad investments
For a profit of $3085.1699999999983 and a portfolio multiple of 1.16x


**Model Architecure 2**

In [291]:
evaluate_model_profit(precision_2_1, recall_2_1, total_target, investment_amount)

You made $13875.0 in good investments
But lost $9359.7 in bad investments
For a profit of $4515.299999999999 and a portfolio multiple of 1.48x


In [292]:
evaluate_model_profit(precision_2_2, recall_2_2, total_target, investment_amount)

You made $5812.5 in good investments
But lost $3279.04 in bad investments
For a profit of $2533.46 and a portfolio multiple of 1.77x


**Model Architecure 3**

In [293]:
evaluate_model_profit(precision_3_1, recall_3_1, total_target, investment_amount)

You made $21300.0 in good investments
But lost $16071.87 in bad investments
For a profit of $5228.129999999999 and a portfolio multiple of 1.33x


In [294]:
evaluate_model_profit(precision_3_2, recall_3_2, total_target, investment_amount)

You made $12000.0 in good investments
But lost $9028.52 in bad investments
For a profit of $2971.4799999999996 and a portfolio multiple of 1.33x


In [295]:
evaluate_model_profit(precision_3_3, recall_3_3, total_target, investment_amount)

You made $6900.0 in good investments
But lost $4490.05 in bad investments
For a profit of $2409.95 and a portfolio multiple of 1.54x


Due to the scarcity of the target class in our data and the element of randomness in neural networks

We validated our process by training and testing on multiple data types to ensure it can recognize patterns in any given data

and we did an aggregate of the evaluation metrics to provide a more honest range of how the model may perform infont of real world data

Our best aggregate model architecture for overall profit was model 3 with $26000, however youll notice the multiple is low, this strategy is higher risk and requires a lot of capital.

A more conservative but highly profitable model would be model 1 with a multiple of 1.74x and profit of $16000

Our best individual model in terms of precision was 'model2_8' at a 0.85 threshold. Let's take a look at the profitibility for that model:

In [296]:
precision_2_8 = 6 / 37
recall_2_8 = 6 / 166
evaluate_model_profit(precision_2_8, recall_2_8, total_target, investment_amount)

You made $9000.0 in good investments
But lost $3100.0 in bad investments
For a profit of $5900.0 and a portfolio multiple of 2.9x


Our best model has a investment porfolio multiple of 2.9x and a profit over $5900

These results far surpass our best base model (Optimized Random Forest)

Over the course of a year we have identified 665 target class investments... 

Theoretically our test data set of 166 target class tokens represents 3-4 months of investing